# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
r = requests.get(url)
data = r.json()
df = pd.DataFrame(data['items'])
df_rj = df[(df['tipo_interessado'] == 'Município') & (df['uf'] == 'RJ')].copy()
df_rj.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
9,6363,Município,Rio Claro,3304409,RJ,None,Regularizado,17944.000328/2008-34,2011-06-01T00:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,6733230.99,0,0,22/07/2011
56,7907,Município,Mesquita,3302858,RJ,None,Deferido,17944.000675/2010-81,2010-09-06T00:00:00Z,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5106563.00,1,0,20/09/2010
109,41862,Município,Santa Maria Madalena,3304607,RJ,PVL02.009405/2019-10,Em retificação pelo interessado,17944.100701/2020-42,2020-02-19T13:20:58Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4770000.00,0,0,06/04/2020
231,5290,Município,Casimiro de Abreu,3301306,RJ,None,Deferido,17944.000078/2014-81,2014-11-28T00:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15024627.28,1,0,12/12/2014
242,7596,Município,Barra Mansa,3300407,RJ,None,Regularizado,17944.000617/2007-52,2007-06-13T00:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,13505401.39,1,0,15/06/2007


In [5]:
# 2) Seu código aqui
frequencias = df_rj['status'].value_counts()
frequencias.head(3)

status
Deferido        25
Arquivado       13
Regularizado     7
Name: count, dtype: int64

In [6]:
# 3) Seu código aqui
df_rj['ano_status'] = df_rj['data_status'].str.slice(6, 10)
df_rj[['data_status', 'ano_status']].head()

,data_status,ano_status
9,22/07/2011,2011
56,20/09/2010,2010
109,06/04/2020,2020
231,12/12/2014,2014
242,15/06/2007,2007


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [ ]:
#1) Seu código aqui
def consultar_pvl(uf: str, tipo: str):
     
     df_dados = df[(df['uf'] == uf) & (df['tipo_interessado'] == tipo)]

     return df_dados

In [ ]:
# 2) Seu código aqui
df_mg = consultar_pvl('MG', 'Estado')
df_mg[df_mg['status'] == 'Arquivado por decurso de prazo'].shape[0]

In [ ]:
# 3) Seu código aqui
df_ba_municipio = consultar_pvl('BA', 'Município')
df_ba_municipio_deferido = df_ba_municipio[df_ba_municipio['status'] == 'Deferido']
df_ba_municipio_deferido['interessado'].value_counts().idxmax()

In [ ]:
# 4) Seu código aqui
df_ba= consultar_pvl('BA', 'Estado')
df_ba.to_csv("solicitacoes_bahia.csv", index=False)